In [1]:
import json
import requests
from time import time, sleep
from datetime import datetime
import os

In [2]:
exchange = 'bfnx'
exchange_path = exchange + '/'
if not os.path.exists(exchange_path): os.makedirs(exchange)

In [3]:
instruments_data = requests.get("https://reference-data-api.kaiko.io/v1/instruments")
idata = instruments_data.json()['data']

instruments = []
bqc = []
currencies = {}
idx = 0
for k, item in enumerate(idata):
    if item['exchange_code'] == exchange:
        x = item['class']
        y = item['exchange_pair_code'].lower()
        base = item['base_asset']
        quote = item['quote_asset']
        bqc.append((base, quote, y))
        instruments.append(y)
        if base not in currencies.keys():
            currencies[str(base)] = idx
            idx += 1
        if quote not in currencies:
            currencies[str(quote)] = idx
            idx += 1
            


In [9]:
exchange_path

'bfnx/'

In [8]:
with open(exchange_path + 'currencies.csv', 'w') as file:
    for currency, val in currencies.items():
        file.write("{},{}\n".format(currency, val))

In [29]:
with open(exchange_path + 'instruments.csv','w') as file:
    file.write('base, quoted, code\n')
    for row in bqc:
        base, quoted, code = row
        file.write("{}, {}, {}\n".format(base, quoted, code))

In [43]:
pairs = []
for base, quote, code in bqc:
    pairs.append((currencies[str(base)], currencies[str(quote)]))
pairs

[(0, 1),
 (0, 2),
 (3, 4),
 (3, 1),
 (3, 2),
 (5, 4),
 (5, 1),
 (5, 2),
 (6, 4),
 (6, 1),
 (6, 2),
 (7, 4),
 (7, 2),
 (7, 8),
 (9, 4),
 (9, 2),
 (9, 8),
 (10, 4),
 (10, 1),
 (10, 2),
 (11, 1),
 (11, 2),
 (12, 4),
 (12, 1),
 (12, 2),
 (13, 4),
 (13, 1),
 (13, 2),
 (14, 4),
 (14, 1),
 (14, 2),
 (15, 4),
 (15, 1),
 (15, 2),
 (16, 4),
 (16, 2),
 (16, 8),
 (17, 4),
 (17, 1),
 (17, 2),
 (18, 1),
 (18, 2),
 (19, 4),
 (19, 2),
 (20, 4),
 (20, 1),
 (20, 2),
 (21, 4),
 (21, 2),
 (22, 4),
 (22, 2),
 (23, 4),
 (23, 1),
 (23, 2),
 (24, 4),
 (24, 2),
 (25, 4),
 (25, 2),
 (26, 4),
 (26, 2),
 (27, 4),
 (27, 1),
 (27, 2),
 (28, 1),
 (28, 2),
 (29, 4),
 (29, 2),
 (30, 4),
 (30, 2),
 (31, 4),
 (31, 2),
 (4, 32),
 (4, 8),
 (4, 33),
 (4, 34),
 (4, 2),
 (4, 8),
 (4, 35),
 (36, 4),
 (36, 2),
 (37, 4),
 (37, 2),
 (38, 4),
 (38, 1),
 (38, 2),
 (39, 4),
 (39, 1),
 (39, 2),
 (40, 4),
 (40, 2),
 (41, 4),
 (41, 1),
 (41, 2),
 (42, 1),
 (42, 2),
 (43, 1),
 (43, 2),
 (44, 4),
 (44, 1),
 (44, 2),
 (45, 4),
 (45, 1),


In [45]:
pairs[0][0]

0

In [48]:
with open(exchange_path + 'pairs.csv', 'w') as file:
    for pair in pairs:
        file.write("{}, {}\n".format(*pair))



In [9]:
kaiko = 'https://us.market-api.kaiko.io/v1/data/trades.latest/exchanges/{}/{}/{}/trades?start_time={}&page_size={}'
header = {"X-Api-Key": "p1s05cqpcv2q862f4x6ngz74tzazll3j"}
instrument_class="spot"
start_time = '2019-06-16T00:00:00.000Z'
max_page_limit = 100000

In [25]:
from datetime import datetime
utc_time = lambda x: datetime.utcfromtimestamp(x / 1000)
def write_rows_to_file(file, data):
    for row in data:
        file.write("{}, {}, {}, {}, {}\n".format(str(row['timestamp']),
                                                 str(row['trade_id']),
                                                 str(row['price']),
                                                 str(row['amount']),
                                                 str(row['taker_side_sell'])))
        lastest_time = row['timestamp']
    return lastest_time

In [26]:
for kth_pair, row in enumerate(bqc):
    base, quoted, pair = row
    pair = base + '-' + quoted
    print(kth_pair, base, quoted, pair)
    next_url = kaiko.format(exchange, instrument_class, pair, start_time, 10000)

    file = open(exchange_path' + pair + ".csv", 'w')
    file.write('timestamp, trade_id, price, amount, taker_side_sell\n')

    k = 1
    while next_url != "":
        tmp = requests.get(next_url, headers = header)
        response = tmp.json()
        data = response['data']
        if len(data) > 0:
            if 'next_url' in response.keys():
                next_url = response['next_url']
            time_stamp = write_rows_to_file(file, data)
            print("----- {}: k = {}, time = {}".format(pair, k, utc_time(time_stamp)))
            k = k + 1
        if 'next_url' not in response.keys(): next_url = ''
    file.close()

0 abyss eth abyss-eth
----- abyss-eth: k = 1, time = 2019-09-23 16:36:36
1 abyss usd abyss-usd
----- abyss-usd: k = 1, time = 2019-09-22 18:50:54
2 agi btc agi-btc
----- agi-btc: k = 1, time = 2019-09-23 22:38:04
3 agi eth agi-eth
----- agi-eth: k = 1, time = 2019-09-23 09:59:12
4 agi usd agi-usd
----- agi-usd: k = 1, time = 2019-09-19 08:16:08
5 aid btc aid-btc
----- aid-btc: k = 1, time = 2019-09-23 15:07:37
6 aid eth aid-eth
----- aid-eth: k = 1, time = 2019-09-23 02:13:42
7 aid usd aid-usd
----- aid-usd: k = 1, time = 2019-09-20 18:51:46
8 aion btc aion-btc
----- aion-btc: k = 1, time = 2019-09-23 21:49:05
9 aion eth aion-eth
----- aion-eth: k = 1, time = 2019-09-23 18:43:03
10 aion usd aion-usd
----- aion-usd: k = 1, time = 2019-09-23 21:33:31
11 algo btc algo-btc
----- algo-btc: k = 1, time = 2019-08-06 08:13:06
----- algo-btc: k = 2, time = 2019-09-23 22:33:18
12 algo usd algo-usd
----- algo-usd: k = 1, time = 2019-09-23 20:33:46
13 algo usdt algo-usdt
----- algo-usdt: k = 1, ti

----- bbn-eth: k = 1, time = 2019-09-21 02:14:26
41 bbn usd bbn-usd
----- bbn-usd: k = 1, time = 2019-09-22 00:29:09
42 bitcsh btc bitcsh-btc
43 bitcsh usd bitcsh-usd
44 bch btc bch-btc
45 bch eth bch-eth
46 bch usd bch-usd
47 bci btc bci-btc
----- bci-btc: k = 1, time = 2019-09-21 13:13:25
48 bci usd bci-usd
----- bci-usd: k = 1, time = 2019-07-13 01:59:09
----- bci-usd: k = 2, time = 2019-07-28 06:27:17
----- bci-usd: k = 3, time = 2019-09-23 22:11:42
49 bcu btc bcu-btc
50 bcu usd bcu-usd
51 bft btc bft-btc
----- bft-btc: k = 1, time = 2019-09-23 17:24:17
52 bft eth bft-eth
----- bft-eth: k = 1, time = 2019-09-23 17:27:47
53 bft usd bft-usd
----- bft-usd: k = 1, time = 2019-09-23 22:45:37
54 bfx btc bfx-btc
55 bfx usd bfx-usd
56 bg1 btc bg1-btc
57 bg1 usd bg1-usd
58 bg2 btc bg2-btc
59 bg2 usd bg2-usd
60 bnt btc bnt-btc
----- bnt-btc: k = 1, time = 2019-09-23 19:03:48
61 bnt eth bnt-eth
----- bnt-eth: k = 1, time = 2019-09-08 04:24:08
62 bnt usd bnt-usd
----- bnt-usd: k = 1, time = 20

----- btc-usdt: k = 28, time = 2019-09-23 22:51:47
73 btc gbp btc-gbp
----- btc-gbp: k = 1, time = 2019-06-23 19:18:15
----- btc-gbp: k = 2, time = 2019-06-27 23:14:03
----- btc-gbp: k = 3, time = 2019-07-03 05:12:05
----- btc-gbp: k = 4, time = 2019-07-09 18:24:18
----- btc-gbp: k = 5, time = 2019-07-12 11:32:54
----- btc-gbp: k = 6, time = 2019-07-20 06:02:33
----- btc-gbp: k = 7, time = 2019-08-06 10:48:01
----- btc-gbp: k = 8, time = 2019-08-19 22:40:37
----- btc-gbp: k = 9, time = 2019-09-01 21:34:59
----- btc-gbp: k = 10, time = 2019-09-21 14:17:58
----- btc-gbp: k = 11, time = 2019-09-23 22:57:59
74 btc jpy btc-jpy
----- btc-jpy: k = 1, time = 2019-06-24 03:16:44
----- btc-jpy: k = 2, time = 2019-06-27 00:36:28
----- btc-jpy: k = 3, time = 2019-06-28 02:32:14
----- btc-jpy: k = 4, time = 2019-07-01 13:08:48
----- btc-jpy: k = 5, time = 2019-07-03 22:28:43
----- btc-jpy: k = 6, time = 2019-07-07 01:02:34
----- btc-jpy: k = 7, time = 2019-07-10 14:25:25
----- btc-jpy: k = 8, time 

----- btc-usd: k = 131, time = 2019-06-30 10:21:39
----- btc-usd: k = 132, time = 2019-06-30 13:49:04
----- btc-usd: k = 133, time = 2019-06-30 14:30:21
----- btc-usd: k = 134, time = 2019-06-30 16:41:51
----- btc-usd: k = 135, time = 2019-06-30 19:52:57
----- btc-usd: k = 136, time = 2019-06-30 22:48:54
----- btc-usd: k = 137, time = 2019-07-01 00:05:13
----- btc-usd: k = 138, time = 2019-07-01 02:45:16
----- btc-usd: k = 139, time = 2019-07-01 05:57:09
----- btc-usd: k = 140, time = 2019-07-01 08:32:54
----- btc-usd: k = 141, time = 2019-07-01 12:25:29
----- btc-usd: k = 142, time = 2019-07-01 14:00:42
----- btc-usd: k = 143, time = 2019-07-01 15:06:35
----- btc-usd: k = 144, time = 2019-07-01 15:55:47
----- btc-usd: k = 145, time = 2019-07-01 18:14:09
----- btc-usd: k = 146, time = 2019-07-01 20:02:24
----- btc-usd: k = 147, time = 2019-07-01 21:44:05
----- btc-usd: k = 148, time = 2019-07-02 00:46:48
----- btc-usd: k = 149, time = 2019-07-02 04:27:24
----- btc-usd: k = 150, time = 

----- btc-usd: k = 292, time = 2019-07-18 14:45:09
----- btc-usd: k = 293, time = 2019-07-18 15:03:48
----- btc-usd: k = 294, time = 2019-07-18 15:46:27
----- btc-usd: k = 295, time = 2019-07-18 16:33:07
----- btc-usd: k = 296, time = 2019-07-18 18:48:22
----- btc-usd: k = 297, time = 2019-07-18 21:22:23
----- btc-usd: k = 298, time = 2019-07-19 00:36:19
----- btc-usd: k = 299, time = 2019-07-19 04:50:16
----- btc-usd: k = 300, time = 2019-07-19 08:19:10
----- btc-usd: k = 301, time = 2019-07-19 09:56:49
----- btc-usd: k = 302, time = 2019-07-19 13:03:07
----- btc-usd: k = 303, time = 2019-07-19 14:46:35
----- btc-usd: k = 304, time = 2019-07-19 20:00:25
----- btc-usd: k = 305, time = 2019-07-20 00:44:52
----- btc-usd: k = 306, time = 2019-07-20 05:30:10
----- btc-usd: k = 307, time = 2019-07-20 11:00:43
----- btc-usd: k = 308, time = 2019-07-20 13:57:02
----- btc-usd: k = 309, time = 2019-07-20 16:57:45
----- btc-usd: k = 310, time = 2019-07-20 18:01:46
----- btc-usd: k = 311, time = 

...

----- ltc-usd: k = 36, time = 2019-07-04 16:11:31
----- ltc-usd: k = 37, time = 2019-07-05 11:07:23
----- ltc-usd: k = 38, time = 2019-07-06 18:26:51
----- ltc-usd: k = 39, time = 2019-07-08 09:05:24
----- ltc-usd: k = 40, time = 2019-07-09 07:51:06
----- ltc-usd: k = 41, time = 2019-07-10 08:24:22
----- ltc-usd: k = 42, time = 2019-07-10 15:13:02
----- ltc-usd: k = 43, time = 2019-07-10 19:00:24
----- ltc-usd: k = 44, time = 2019-07-11 04:27:20
----- ltc-usd: k = 45, time = 2019-07-11 15:22:43
----- ltc-usd: k = 46, time = 2019-07-12 07:19:27
----- ltc-usd: k = 47, time = 2019-07-13 10:17:23
----- ltc-usd: k = 48, time = 2019-07-14 01:02:38
----- ltc-usd: k = 49, time = 2019-07-14 12:26:07
----- ltc-usd: k = 50, time = 2019-07-14 21:41:03
----- ltc-usd: k = 51, time = 2019-07-15 05:10:45
----- ltc-usd: k = 52, time = 2019-07-15 20:14:46
----- ltc-usd: k = 53, time = 2019-07-16 16:16:40
----- ltc-usd: k = 54, time = 2019-07-16 18:21:17
----- ltc-usd: k = 55, time = 2019-07-17 09:30:49


----- neo-usd: k = 21, time = 2019-07-10 14:21:23
----- neo-usd: k = 22, time = 2019-07-12 07:48:13
----- neo-usd: k = 23, time = 2019-07-13 21:27:16
----- neo-usd: k = 24, time = 2019-07-14 15:50:03
----- neo-usd: k = 25, time = 2019-07-16 11:00:36
----- neo-usd: k = 26, time = 2019-07-17 14:18:38
----- neo-usd: k = 27, time = 2019-07-18 15:24:50
----- neo-usd: k = 28, time = 2019-07-19 14:36:11
----- neo-usd: k = 29, time = 2019-07-21 09:04:43
----- neo-usd: k = 30, time = 2019-07-23 07:07:31
----- neo-usd: k = 31, time = 2019-07-24 20:28:10
----- neo-usd: k = 32, time = 2019-07-27 11:28:29
----- neo-usd: k = 33, time = 2019-07-31 02:02:46
----- neo-usd: k = 34, time = 2019-08-05 10:32:26
----- neo-usd: k = 35, time = 2019-08-09 10:25:11
----- neo-usd: k = 36, time = 2019-08-13 12:08:46
----- neo-usd: k = 37, time = 2019-08-17 02:56:23
----- neo-usd: k = 38, time = 2019-08-23 13:10:21
----- neo-usd: k = 39, time = 2019-08-29 06:06:43
----- neo-usd: k = 40, time = 2019-09-06 01:46:01


----- rrt-usd: k = 1, time = 2019-09-23 21:32:54
319 rte eth rte-eth
----- rte-eth: k = 1, time = 2019-09-20 13:14:29
320 rte usd rte-usd
----- rte-usd: k = 1, time = 2019-08-28 07:47:03
321 san btc san-btc
----- san-btc: k = 1, time = 2019-09-23 21:26:22
322 san eth san-eth
----- san-eth: k = 1, time = 2019-09-23 21:04:39
323 san usd san-usd
----- san-usd: k = 1, time = 2019-07-05 08:12:23
----- san-usd: k = 2, time = 2019-08-07 15:43:58
----- san-usd: k = 3, time = 2019-09-23 23:10:49
324 scrl eth scrl-eth
----- scrl-eth: k = 1, time = 2019-08-22 10:53:17
----- scrl-eth: k = 2, time = 2019-09-23 21:56:54
325 scrl usd scrl-usd
----- scrl-usd: k = 1, time = 2019-07-17 21:52:18
----- scrl-usd: k = 2, time = 2019-09-16 05:55:16
----- scrl-usd: k = 3, time = 2019-09-23 21:33:31
326 seer btc seer-btc
----- seer-btc: k = 1, time = 2019-09-23 21:32:34
327 seer eth seer-eth
----- seer-eth: k = 1, time = 2019-09-21 21:18:00
328 seer usd seer-usd
----- seer-usd: k = 1, time = 2019-09-23 21:21:4

382 ven eth ven-eth
383 ven usd ven-usd
384 vet btc vet-btc
----- vet-btc: k = 1, time = 2019-06-24 01:21:00
----- vet-btc: k = 2, time = 2019-06-30 06:27:00
----- vet-btc: k = 3, time = 2019-07-06 21:48:30
----- vet-btc: k = 4, time = 2019-07-13 08:06:30
----- vet-btc: k = 5, time = 2019-07-19 19:47:00
----- vet-btc: k = 6, time = 2019-07-26 00:20:45
----- vet-btc: k = 7, time = 2019-08-01 01:40:15
----- vet-btc: k = 8, time = 2019-08-07 17:15:45
----- vet-btc: k = 9, time = 2019-08-14 14:56:00
----- vet-btc: k = 10, time = 2019-08-20 23:13:46
----- vet-btc: k = 11, time = 2019-08-27 09:40:45
----- vet-btc: k = 12, time = 2019-09-03 01:19:16
----- vet-btc: k = 13, time = 2019-09-09 15:39:16
----- vet-btc: k = 14, time = 2019-09-23 22:48:01
385 vet eth vet-eth
----- vet-eth: k = 1, time = 2019-06-25 20:40:10
----- vet-eth: k = 2, time = 2019-07-05 17:23:34
----- vet-eth: k = 3, time = 2019-07-16 20:57:11
----- vet-eth: k = 4, time = 2019-07-27 20:51:40
----- vet-eth: k = 5, time = 2019

----- xrp-usd: k = 19, time = 2019-06-23 21:33:56
----- xrp-usd: k = 20, time = 2019-06-24 00:18:37
----- xrp-usd: k = 21, time = 2019-06-24 07:16:19
----- xrp-usd: k = 22, time = 2019-06-24 20:39:51
----- xrp-usd: k = 23, time = 2019-06-25 11:05:06
----- xrp-usd: k = 24, time = 2019-06-25 20:55:52
----- xrp-usd: k = 25, time = 2019-06-26 05:34:36
----- xrp-usd: k = 26, time = 2019-06-26 17:34:27
----- xrp-usd: k = 27, time = 2019-06-26 20:56:49
----- xrp-usd: k = 28, time = 2019-06-27 00:24:02
----- xrp-usd: k = 29, time = 2019-06-27 06:30:40
----- xrp-usd: k = 30, time = 2019-06-27 09:12:53
----- xrp-usd: k = 31, time = 2019-06-27 16:25:58
----- xrp-usd: k = 32, time = 2019-06-27 21:15:16
----- xrp-usd: k = 33, time = 2019-06-28 08:33:35
----- xrp-usd: k = 34, time = 2019-06-28 23:57:16
----- xrp-usd: k = 35, time = 2019-06-29 15:24:12
----- xrp-usd: k = 36, time = 2019-06-30 01:06:20
----- xrp-usd: k = 37, time = 2019-06-30 14:38:50
----- xrp-usd: k = 38, time = 2019-07-01 01:18:33
